# 搜索增强生成进阶（RAG，Retrieval-Augmented Generation）

## 💡 这节课会带给你


1. RAG 中的进阶知识
1. 如何优化RAG系统
1. Gradio 上传文件功能

开始上课！


## 一、实战 RAG 系统的进阶知识


### 1.1、文本分割的粒度

![](./rag-document-split.png)


**缺陷**

1. 粒度太大可能导致检索不精准，粒度太小可能导致信息不全面
2. 问题的答案可能跨越两个片段


In [4]:
!pip uninstall gradio -y
!pip install chromadb
!pip install pydantic==1.9.0
# ISSUE HERE: https://github.com/chroma-core/chroma/issues/774

Found existing installation: gradio 4.36.1
Uninstalling gradio-4.36.1:
  Successfully uninstalled gradio-4.36.1
  Using cached pydantic-1.10.16-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (151 kB)
  Using cached fastapi-0.85.1-py3-none-any.whl.metadata (24 kB)
  Using cached starlette-0.20.4-py3-none-any.whl.metadata (5.5 kB)
Using cached fastapi-0.85.1-py3-none-any.whl (55 kB)
Using cached starlette-0.20.4-py3-none-any.whl (63 kB)
Using cached pydantic-1.10.16-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.4
    Uninstalling pydantic-2.7.4:
      Successfully uninstalled pydantic-2.7.4
  Attempting uninstall: starlette
    Found existing installation: starlette 0.37.2
    Uninstalling starlette-0.37.2:
      Successfully uninstalled starlette-0.37.2
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.111.0
    Uninstalling fastapi-0.111.0:
    

In [1]:
import chromadb
from chromadb.config import Settings


class MyVectorDBConnector:
    def __init__(self, collection_name, embedding_fn):
        chroma_client = chromadb.Client(Settings(allow_reset=True))

        # 为了演示，实际不需要每次 reset()
        chroma_client.reset()

        # 创建一个 collection
        self.collection = chroma_client.get_or_create_collection(name=collection_name)
        self.embedding_fn = embedding_fn

    def add_documents(self, documents):
        '''向 collection 中添加文档与向量'''
        self.collection.add(
            embeddings=self.embedding_fn(documents),  # 每个文档的向量
            documents=documents,  # 文档的原文
            ids=[f"id{i}" for i in range(len(documents))]  # 每个文档的 id
        )

    def search(self, query, top_n):
        '''检索向量数据库'''
        results = self.collection.query(
            query_embeddings=self.embedding_fn([query]),
            n_results=top_n
        )
        return results

In [2]:
def get_embeddings(texts, model="text-embedding-ada-002",dimensions=None):
    '''封装 OpenAI 的 Embedding 模型接口，文档地址 https://platform.openai.com/docs/guides/embeddings/what-are-embeddings'''
    if model == "text-embedding-ada-002":
        dimensions = None
    if dimensions:
        data = client.embeddings.create(input=texts, model=model, dimensions=dimensions).data
    else:
        data = client.embeddings.create(input=texts, model=model).data
    return [x.embedding for x in data]

In [3]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer


def extract_text_from_pdf(filename, page_numbers=None, min_line_length=1):
    '''从 PDF 文件中（按指定页码）提取文字'''
    paragraphs = []
    buffer = ''
    full_text = ''
    # 提取全部文本
    for i, page_layout in enumerate(extract_pages(filename)):
        # 如果指定了页码范围，跳过范围外的页
        if page_numbers is not None and i not in page_numbers:
            continue
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                full_text += element.get_text() + '\n'
                
    # 按空行分隔，将文本重新组织成段落
    lines = full_text.split('\n')
    for text in lines:
        if len(text) >= min_line_length:
            buffer += (' '+text) if not text.endswith('-') else text.strip('-')
        elif buffer:
            paragraphs.append(buffer)
            buffer = ''
    if buffer:
        paragraphs.append(buffer)
    return paragraphs

In [4]:
# 为了演示方便，我们只取两页（第一章）
paragraphs = extract_text_from_pdf(
    "llama2.pdf", 
    page_numbers=[2, 3], 
    min_line_length=10
)

In [5]:
from openai import OpenAI
import os
# 加载环境变量
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # 读取本地 .env 文件，里面定义了 OPENAI_API_KEY

client = OpenAI()

In [6]:
class RAG_Bot:
    def __init__(self, vector_db, llm_api, n_results=2):
        self.vector_db = vector_db
        self.llm_api = llm_api
        self.n_results = n_results

    def chat(self, user_query):
        # 1. 检索
        search_results = self.vector_db.search(user_query, self.n_results)

        # 2. 构建 Prompt
        prompt = build_prompt(
            prompt_template, info=search_results['documents'][0], query=user_query)

        # 3. 调用 LLM
        response = self.llm_api(prompt)
        return response

In [7]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''封装 openai 接口'''
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
    )
    return response.choices[0].message.content

In [8]:
# 创建一个向量数据库对象
vector_db = MyVectorDBConnector("demo_text_split", get_embeddings)
# 向向量数据库中添加文档
vector_db.add_documents(paragraphs)

# 创建一个RAG机器人
bot = RAG_Bot(
    vector_db,
    llm_api=get_completion
)

In [9]:
def build_prompt(prompt_template, **kwargs):
    '''将 Prompt 模板赋值'''
    prompt = prompt_template
    for k, v in kwargs.items():
        if isinstance(v, str):
            val = v
        elif isinstance(v, list) and all(isinstance(elem, str) for elem in v):
            val = '\n'.join(v)
        else:
            val = str(v)
        prompt = prompt.replace(f"__{k.upper()}__", val)
    return prompt

In [10]:
prompt_template = """
你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

已知信息:
__INFO__

用户问：
__QUERY__

请用中文回答用户问题。
"""

In [11]:
user_query = "llama 2可以商用吗？"
# user_query="llama 2 chat有多少参数"
search_results = vector_db.search(user_query, 2)

for doc in search_results['documents'][0]:
    print(doc+"\n")

print("====回复====")
bot.chat(user_query)

 We believe that the open release of LLMs, when done safely, will be a net benefit to society. Like all LLMs, Llama 2 is a new technology that carries potential risks with use (Bender et al., 2021b; Weidinger et al., 2021; Solaiman et al., 2023). Testing conducted to date has been in English and has not — and could not — cover all scenarios. Therefore, before deploying any applications of Llama 2-Chat, developers should perform safety testing and tuning tailored to their specific applications of the model. We provide a responsible use guide¶ and code examples‖ to facilitate the safe deployment of Llama 2 and Llama 2-Chat. More details of our responsible release strategy can be found in Section 5.3.

 1. Llama 2, an updated version of Llama 1, trained on a new mix of publicly available data. We also increased the size of the pretraining corpus by 40%, doubled the context length of the model, and adopted grouped-query attention (Ainslie et al., 2023). We are releasing variants of Llama 2

'我无法回答您的问题。'

In [12]:
for p in paragraphs:
    print(p+"\n")

 Figure 1: Helpfulness human evaluation results for Llama 2-Chat compared to other open-source and closed-source models. Human raters compared model generations on ~4k prompts consisting of both single and multi-turn prompts. The 95% confidence intervals for this evaluation are between 1% and 2%. More details in Section 3.4.2. While reviewing these results, it is important to note that human evaluations can be noisy due to limitations of the prompt set, subjectivity of the review guidelines, subjectivity of individual raters, and the inherent difficulty of comparing generations.

 Figure 2: Win-rate % for helpfulness andsafety between commercial-licensed baselines and Llama 2-Chat, according to GPT 4. To complement the human evaluation, we used a more capable model, not subject to our own guidance. Green area indicates our model is better according to GPT-4. To remove ties, we used win/(win + loss). The orders in which the model responses are presented to GPT-4 are randomly swapped to 

**改进**: 按一定粒度，部分重叠式的切割文本，使上下文更完整

![](./rag-overlap.jpg)


In [13]:
from nltk.tokenize import sent_tokenize
import json


def split_text(paragraphs, chunk_size=300, overlap_size=100):
    '''按指定 chunk_size 和 overlap_size 交叠割文本'''
    sentences = [s.strip() for p in paragraphs for s in sent_tokenize(p)]
    chunks = []
    i = 0
    while i < len(sentences):
        chunk = sentences[i]
        overlap = ''
        prev_len = 0
        prev = i - 1
        # 向前计算重叠部分
        while prev >= 0 and len(sentences[prev])+len(overlap) <= overlap_size:
            overlap = sentences[prev] + ' ' + overlap
            prev -= 1
        chunk = overlap+chunk
        next = i + 1
        # 向后计算当前chunk
        while next < len(sentences) and len(sentences[next])+len(chunk) <= chunk_size:
            chunk = chunk + ' ' + sentences[next]
            next += 1
        chunks.append(chunk)
        i = next
    return chunks

<div class="alert alert-info">
此处 sent_tokenize 为针对英文的实现，针对中文的实现请参考 chinese_utils.py
</div>

In [14]:
chunks = split_text(paragraphs, 300, 100)

In [15]:
# 创建一个向量数据库对象
vector_db = MyVectorDBConnector("demo_text_split", get_embeddings)
# 向向量数据库中添加文档
vector_db.add_documents(chunks)
# 创建一个RAG机器人
bot = RAG_Bot(
    vector_db,
    llm_api=get_completion
)

In [16]:
user_query = "llama 2可以商用吗？"
# user_query="llama 2 chat有多少参数"

search_results = vector_db.search(user_query, 2)
for doc in search_results['documents'][0]:
    print(doc+"\n")

response = bot.chat(user_query)
print("====回复====")
print(response)

We are releasing the following models to the general public for research and commercial use‡: 1. Llama 2, an updated version of Llama 1, trained on a new mix of publicly available data.

Llama 2-Chat, a fine-tuned version of Llama 2 that is optimized for dialogue use cases. We release variants of this model with 7B, 13B, and 70B parameters as well. We believe that the open release of LLMs, when done safely, will be a net benefit to society.

====回复====
可以商用。


### 1.2、检索后排序（选）


**问题**: 有时，最合适的答案不一定排在检索的最前面


In [20]:
user_query = "how safe is llama 2"
search_results = vector_db.search(user_query, 5)

for doc in search_results['documents'][0]:
    print(doc+"\n")

response = bot.chat(user_query)
print("====回复====")
print(response)

We believe that the open release of LLMs, when done safely, will be a net benefit to society. Like all LLMs, Llama 2 is a new technology that carries potential risks with use (Bender et al., 2021b; Weidinger et al., 2021; Solaiman et al., 2023).

We also share novel observations we made during the development of Llama 2 and Llama 2-Chat, such as the emergence of tool usage and temporal organization of knowledge. Figure 3: Safety human evaluation results for Llama 2-Chat compared to other open-source and closed source models.

In this work, we develop and release Llama 2, a family of pretrained and fine-tuned LLMs, Llama 2 and Llama 2-Chat, at scales up to 70B parameters. On the series of helpfulness and safety benchmarks we tested, Llama 2-Chat models generally perform better than existing open-source models.

Additionally, these safety evaluations are performed using content standards that are likely to be biased towards the Llama 2-Chat models. We are releasing the following models t

**方案**:

1. 检索时过召回一部分文本
2. 通过一个排序模型对 query 和 document 重新打分排序


<img src="sbert-rerank.png" style="margin-left: 0px" width=600px>




<div class="alert alert-warning">
<b>备注：</b>
<div>由于 huggingface 被墙，我们已经为您准备好了本章相关模型。请点击以下网盘链接进行下载：
    
链接: https://pan.baidu.com/s/1X0kfNKasvWqCLUEEyAvO-Q?pwd=3v6y 提取码: 3v6y </div>
</div>


In [ ]:
!pip install sentence_transformers

使用 CrossEncoder 来作为重新打分排序的模型，CrossEncoder 的文档地址：https://www.sbert.net/examples/applications/cross-encoder/README.html

<img src="./cross-encoder.png" width="600px"></img>

**同时将两个句子传递给 Transformer 网络。它产生一个输出值，表示输入句对的相似性，值越大越相近。**


In [17]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)

/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
user_query = "how safe is llama 2"

scores = model.predict([(user_query, doc)
                       for doc in search_results['documents'][0]])
# 按得分排序
sorted_list = sorted(
    zip(scores, search_results['documents'][0]), key=lambda x: x[0], reverse=True)
for score, doc in sorted_list:
    print(f"{score}\t{doc}\n")

5.598450660705566	Llama 2-Chat, a fine-tuned version of Llama 2 that is optimized for dialogue use cases. We release variants of this model with 7B, 13B, and 70B parameters as well. We believe that the open release of LLMs, when done safely, will be a net benefit to society.

-0.4974502623081207	We are releasing the following models to the general public for research and commercial use‡: 1. Llama 2, an updated version of Llama 1, trained on a new mix of publicly available data.



### 1.3、混合检索（Hybrid Search）（选）

在**实际生产**中，传统的关键字检索（稀疏表示）与向量检索（稠密表示）各有优劣。

举个具体例子，比如文档中包含很长的专有名词，关键字检索往往更精准而向量检索容易引入概念混淆。

In [19]:
import numpy as np
from numpy import dot
from numpy.linalg import norm


def cos_sim(a, b):
    '''余弦距离 -- 越大越相似'''
    return dot(a, b)/(norm(a)*norm(b))

In [20]:
# 背景说明：在医学中“小细胞肺癌”和“非小细胞肺癌”是两种不同的癌症

query = "非小细胞肺癌的患者"

documents = [
    "李某患有肺癌，癌细胞已转移",
    "刘某肺癌I期",
    "张某经诊断为非小细胞肺癌III期",
    "小细胞肺癌是肺癌的一种"
]

query_vec = get_embeddings([query])[0]
doc_vecs = get_embeddings(documents)

print("Cosine distance:")
for vec in doc_vecs:
    print(cos_sim(query_vec, vec))

Cosine distance:
0.910667535993348
0.8895478505819983
0.9039165614288258
0.9131441645902687


所以，有时候我们需要结合不同的检索算法，来达到比单一检索算法更优的效果。这就是**混合检索**。

混合检索的核心是，综合文档 $d$ 在不同检索算法下的排序名次（rank），为其生成最终排序。

一个最常用的算法叫 **Reciprocal Rank Fusion（RRF）**

$rrf(d)=\sum_{a\in A}\frac{1}{k+rank_a(d)}$

其中 $A$ 表示所有使用的检索算法的集合，$rank_a(d)$ 表示使用算法 $a$ 检索时，文档 $d$ 的排序，$k$ 是个常数。

很多向量数据库都支持混合检索，比如 [Weaviate](https://weaviate.io/blog/hybrid-search-explained)、[Pinecone](https://www.pinecone.io/learn/hybrid-search-intro/) 等。也可以根据上述原理自己实现。

### 1.3.1、我们手写个简单的例子

1. 基于关键字检索的排序

In [21]:
import time

class MyEsConnector:
    def __init__(self, es_client, index_name, keyword_fn):
        self.es_client = es_client
        self.index_name = index_name
        self.keyword_fn = keyword_fn
    
    def add_documents(self, documents):
        '''文档灌库'''
        if self.es_client.indices.exists(index=self.index_name):
            self.es_client.indices.delete(index=self.index_name)
        self.es_client.indices.create(index=self.index_name)
        actions = [
            {
                "_index": self.index_name,
                "_source": {
                    "keywords": self.keyword_fn(doc),
                    "text": doc,
                    "id": f"doc_{i}"
                }
            }
            for i,doc in enumerate(documents)
        ]
        helpers.bulk(self.es_client, actions)
        time.sleep(1)

    def search(self, query_string, top_n=3):
        '''检索'''
        search_query = {
            "match": {
                "keywords": self.keyword_fn(query_string)
            }
        }
        res = self.es_client.search(index=self.index_name, query=search_query, size=top_n)
        return { 
            hit["_source"]["id"] : {
                "text" : hit["_source"]["text"],
                "rank" : i,
            }
            for i, hit in enumerate(res["hits"]["hits"])
        }

In [22]:
!pip install jieba

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
import jieba
from elasticsearch7 import Elasticsearch, helpers

In [24]:
from chinese_utils import to_keywords # 使用中文的关键字提取函数

es = Elasticsearch(
    hosts=['http://localhost:9200'],  # 服务地址与端口
    # http_auth=("elastic", "FKaB1Jpz0Rlw0l6G"),  # 用户名，密码
)

# 创建 ES 连接器
es_connector = MyEsConnector(es, "demo_es_rrf", to_keywords)

# 文档灌库
es_connector.add_documents(documents)

# 关键字检索
keyword_search_results = es_connector.search(query, 3)

print(keyword_search_results)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.814 seconds.
Prefix dict has been built successfully.


{'doc_2': {'text': '张某经诊断为非小细胞肺癌III期', 'rank': 0}, 'doc_0': {'text': '李某患有肺癌，癌细胞已转移', 'rank': 1}, 'doc_3': {'text': '小细胞肺癌是肺癌的一种', 'rank': 2}}


2. 基于向量检索的排序

In [25]:
# 创建向量数据库连接器
vecdb_connector = MyVectorDBConnector("demo_vec_rrf", get_embeddings)

# 文档灌库
vecdb_connector.add_documents(documents)

# 向量检索
vector_search_results = {
    "doc_"+str(documents.index(doc)) : {
        "text" : doc,
        "rank" : i
    }
    for i, doc in enumerate(
        vecdb_connector.search(query, 3)["documents"][0]
    )
} # 把结果转成跟上面关键字检索结果一样的格式

print(vector_search_results)

{'doc_3': {'text': '小细胞肺癌是肺癌的一种', 'rank': 0}, 'doc_0': {'text': '李某患有肺癌，癌细胞已转移', 'rank': 1}, 'doc_2': {'text': '张某经诊断为非小细胞肺癌III期', 'rank': 2}}


3. 基于 RRF 的融合排序

   $rrf(d)=\sum_{a\in A}\frac{1}{k+rank_a(d)}$

In [26]:
def rrf(ranks, k=1):
    ret = {}
    # 遍历每次的排序结果
    for rank in ranks: 
        # 遍历排序中每个元素
        for id, val in rank.items():
            if id not in ret:
                ret[id] = { "score": 0, "text": val["text"] }
            # 计算 RRF 得分
            ret[id]["score"] += 1.0/(k+val["rank"])
    # 按 RRF 得分排序，并返回
    return dict(sorted(ret.items(), key=lambda item: item[1]["score"], reverse=True))

In [27]:
import json

# 融合两次检索的排序结果
reranked = rrf([keyword_search_results,vector_search_results])

print(json.dumps(reranked,indent=4,ensure_ascii=False))

{
    "doc_2": {
        "score": 1.3333333333333333,
        "text": "张某经诊断为非小细胞肺癌III期"
    },
    "doc_3": {
        "score": 1.3333333333333333,
        "text": "小细胞肺癌是肺癌的一种"
    },
    "doc_0": {
        "score": 1.0,
        "text": "李某患有肺癌，癌细胞已转移"
    }
}


### 1.4、RAG-Fusion

RAG-Fusion 就是利用了 RRF 的原理来提升检索的准确性。

<img src="rag-fusion.jpeg" style="margin-left: 0px" width=600px>

原始项目（一段非常简短的演示代码）：https://github.com/Raudaschl/rag-fusion

## 二、向量模型的本地部署


利用 sentence_transformers 的本地部署能力，部署向量模型。

<div class="alert alert-warning">
<b>备注：</b>
<div>由于 huggingface 被墙，我们已经为您准备好了本章相关模型。请点击以下网盘链接进行下载：
    
链接: https://pan.baidu.com/s/1X0kfNKasvWqCLUEEyAvO-Q?pwd=3v6y 提取码: 3v6y </div>
</div>

In [32]:
from sentence_transformers import SentenceTransformer

#model_name = 'BAAI/bge-large-zh-v1.5' #中文
model_name = 'moka-ai/m3e-base' #中英双语，但效果一般

model = SentenceTransformer(model_name)

/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
#query = "国际争端"
query = "global conflicts"

documents = [
    "联合国就苏丹达尔富尔地区大规模暴力事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典“入约”问题进行谈判",
    "日本岐阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营",
    "我国首次在空间站开展舱外辐射生物学暴露实验",
]

query_vec = model.encode(query)

doc_vecs = [
    model.encode(doc)
    for doc in documents
]

print("Cosine distance:")  # 越大越相似
#print(cos_sim(query_vec, query_vec))
for vec in doc_vecs:
    print(cos_sim(query_vec, vec))

Cosine distance:
0.6958814
0.6573525
0.6653428
0.637189
0.69429


<div class="alert alert-info">
<b>扩展阅读：https://github.com/FlagOpen/FlagEmbedding</b>
</div>


<div class="alert alert-success">
<b>划重点：</b>
    <ol>
        <li>不是每个 Embedding 模型都对余弦距离和欧氏距离同时有效</li>
        <li>哪种相似度计算有效要阅读模型的说明（通常都支持余弦距离计算）</li>
    </ol>
</div>


## 三、如何持续改进RAG应用效果？（前延扩展，综合思考）

随着深入使用，你可能发现你的 RAG 应用可能只是能用了，但还有很多问题，比如：

- 问题比较抽象或者概念比较模糊，导致大模型没有准确理解使用者的问题。例如，使用者问“兰州拉面去哪吃？”，使用者本来想问附近有没有卖兰州拉面的店铺；假如知识库搜索“兰州”和“拉面”之后，结果排序靠前的语料是位于“兰州”的拉面馆地址，而大模型告诉用户要去买飞机票或者火车票，就是答非所问了。通常我们用改造问题，让使用者的问题更好理解的策略来回避这种情况。
- 知识库没有检索到问题的答案，这有可能是由于语料数据没有做好整理就存入知识库，或者是检索策略有问题，参数需要调整导致的。比如，在采用“K个最相似文档块”作为回答的知识这个策略中，如果K值比较小，那么最相似的K个文档块中可能并不包含能解答用户问题的有效知识，那么答案很可能就是错误的。例如，作为旅游手册的知识库中有大量文段是 兰州拉面如何制作的菜谱、兰州拉面的产地、兰州有哪些特产等，只有一两条信息描述了附近拉面馆的地址。那么当用户询问“兰州拉面怎么走？”时，知识库检索到的信息可能只是兰州拉面的选材、调味、烹饪方面的信息，而唯独没有检索到前方50米处有一家兰州拉面馆。用户也没有办法获得有效的答案。
- 缺少对答案做兜底验证的机制，假设运气很好，志愿者不仅听懂了游客的问题，也正确查找到了附近最近的两家拉面馆的信息，但是志愿者的回答方式是“向北走200米就到了”。这有可能是一个正确的答案，但不是一个好的答案，实际考察过景区地形后我们可能会发现，志愿者北面是后海，你不太可能穿过湖面去一个地方。实际路径可能是：先向东走50十米，再向北走绕过后海，走到湖对面去，才能走到正确位置。那这个“向北走200米...”的回答，从导航的角度就不能算是准确了。

我们会发现许多改进标准 RAG 框架的方法，下面我们将一起了解这些改进思路。

![](./rag-promote.jpg)

### 3.1 建立评测标准
为了持续改进我们的 RAG 应用，首要任务应当是构建一套严谨的评测指标体系，并邀请业务领域专家作为评测方共同参与评测工作，我们可以设置与我们业务相关的多种问题场景，系统性地检查一个RAG系统反应快不快，回答准不准，有没有理解用户问题的意图等方面。通过科学全面的评测，我们可以了解到系统在哪些地方做得好，哪些地方需要改进，从而帮助开发者让RAG系统更好的服务于业务需求。
RAG系统一般包括检索和生成两个模块，我们做评测时就可以从这两个模块分别入手建立评价标准和实施方法，当然你也可以用最终效果为标准，建立端到端的评测。在评测指标设计上，我们主要考察检索模块的准确性，如准确率、召回率、F1值等等；在生成模块，我们主要考察生成答案的价值，如相关性、真实性等等。
我们可以引入业界认可的一些通用评估策略，比如，你可以参考Ragas提及的评测矩阵指南，你也可以建立一些自己的评测指标，这些评测方法将会有助于你量化和改进每一个子模块的表现。

**现在大家先对Ragas有个印象，后面讲Langchain还会讲这个怎么计算和使用**


![](./rag-promote.jpg)

### 3.2 改造一：提升索引准确率

- **优化文本解析过程**
  
在构建知识库的时候，我们首先需要正确的从文档中提取有效语料。因此，优化文本解析的过程往往对提升RAG的性能有很大帮助。例如，从网页中提取有效信息时，我们需要判断哪些部分应该被去掉（比如页眉页脚标签），哪些部分应该被保留（比如属于网页内容的表格标签）。

- **优化chunk切分模式**
  
Chunk就是数据或信息的一个小片段或者区块。当你在处理大量的文本、数据或知识时，如果你一次性全部交给大模型来阅读和处理，效率是非常低的。所以，我们把它们切分成更小、更易管理的部分，这些部分就是chunk。每个chunk都包含了一些有用的信息，这样当系统根据用户的问题寻找某个知识时，它可以迅速定位到与答案相关信息的chunk，而不是在整个数据库中盲目搜索。因此，通过精心设计的chunk切分策略，系统能够更高效地检索信息，就像图书馆里按照类别和标签整理书籍一样，使得查找特定内容变得容易。优化chunk切分模式，就能加速信息检索、提升回答质量和生成效率。具体方法有很多：

    1. 利用领域知识：针对特定领域的文档，利用领域专有知识进行更精准的切分。例如，在法律文档中识别段落编号、条款作为切分依据。
    2. 基于固定大小切分：比如默认采用128个词或512个词切分为一个chunk，可以快速实现文本分块。缺点是忽略了语义和上下文完整性。
    3. 上下文感知：在切分时考虑前后文关系，避免信息断裂。可以通过保持特定句对或短语相邻，或使用更复杂的算法识别并保留语义完整性。最简单的做法是切分时保留前一句和后一句话。你也可以使用自然语言处理技术识别语义单元，如通过句子相似度计算、主题模型（如LDA）或BERT嵌入聚类来切分文本，确保每个chunk内部语义连贯，减少跨chunk信息依赖。通义实验室提供了一种文本切割模型，输入长文本即可得到切割好的文本块，详情可参考：中文文本分割模型。

以上介绍了一些常见策略，你也可以考虑使用更复杂的切分策略，如围绕关键词切分或者采用动态调整的切分策略等，主要目的是为了保证每个chunk中信息的完整性，更好的服务系统提升检索质量。

- **句子滑动窗口检索**
  
这个策略是通过设置window_size（窗口大小）来调整提取句子的数量，当用户的问题匹配到一个chunk语料块时，通过窗函数提取目标语料块的上下文，而不仅仅是语料块本身，这样来获得更完整的语料上下文信息，提升RAG生成质量。

![](./rag-context.jpg)
图6：句子滑窗检索获取检索到的句子的上下文

- **自动合并检索**

这个策略是将文档分块，建成一棵语料块的树，比如1024切分、512切分、128切分，并构造出一棵文档结构树。当应用作搜索时，如果同一个父节点的多个叶子节点被选中，则返回整个父节点对应的语料块。从而确保与问题相关的语料信息被完整保留下来，从而大幅提升RAG的生成质量。实测发现这个方法比句子滑动窗口检索效果好。

![](./rag-merge.jpg)
图7：自动合并检索的方法，返回父节点文本作为检索结果


- **选择更适合业务的Embedding模型**


经过切分的语料块在提供检索服务之前，我们需要把chunk语料块由原来的文本内容转换为机器可以用于比对计算的一组数字，即变为Embedding向量。我们通过Embedding模型来进行这个转换。但是，由于不同的Embedding模型对于生成Embedding向量质量的影响很大，好的Embedding模型可以提升检索的准确率。
比如，针对中文检索的场景，我们应当选择在中文语料上表现更好的模型。那么针对你的业务场景，你也可以建议你的技术团队做Embedding模型的技术选型，挑选针对你的业务场景表现较好的模型。

- **选择更适合业务的ReRank模型**


除了优化生成向量的质量，我们还需要同时优化做向量排序的ReRank模型，好的ReRank模型会让更贴近用户问题的chunks的排名更靠前。因此，我们也可以挑选能让你的业务应用表现更好的ReRank模型。

- **Raptor 用聚类为文档块建立索引**


还有一类有意思的做法是采用无监督聚类来生成文档索引。这就像通过文档的内容为文档自动建立目录的过程。假如志愿者拿到的文本资料是没有目录的，志愿者一页一页查找资料必然很慢。因此，可以将词条信息聚类，比如按照商店、公园、酒吧、咖啡店、中餐馆、快餐店等方式进行分组，建立目录，再根据汉语拼音字母来排序。这样志愿者来查找信息的时候就可以更快速地进行定位。

![](./raptor.jpg)
图8：Raptor 用聚类为文档块建立索引

### 3.3 改造二：让问题更好理解
我们希望做到能让人们通过口语对话来使用大模型应用。然而，人们在口语化表达自己的目的和意图时，往往会出现一些问题。比如，问题过于简单含糊出现了语义混淆，导致大模型理解错误；问题的要素非常多，而用户又讲得太少，只能在反复对话中不断沟通补全；问题涉及的知识点超出了大模型训练语料，或者知识库的覆盖范围，导致大模型编造了一些信息来回答等等。所以，我们期望能在用户提问的环节进行介入，让大模型能更好的理解用户的问题。针对这个问题进行尝试的论文很多，提供了很多有意思的实现思路，如Multi-Query、RAG-Fusion、Decomposition、Step-back、HyDE等等，我们简要讲解一下这些方法的思路。

- **Enrich 完善用户问题**
  
我们首先介绍一种比较容易想到的思路，让大模型来完善用户的原始问题，产生一个更利于系统理解的完善后的用户问题，再让后续的系统去执行用户的需求。通过用大模型对用户的问题进行专业化改写，特别是加入了知识库的支持，我们可以生成更专业的问题。下图展现了一种理想的对用户问题的Enrich流程。我们通过多轮对话逐步确认用户需求。

    - 一种理想的通过多轮对话补全需求的方案。该设想是通过大模型多次主动与用户沟通，不断收集信息，完善对用户真实意图的理解，补全执行用户需求所需的各项参数。如下图所示。

![](./rag-qa-flow.jpg)
图9：通过多轮对话完善用户问题的工作流

以下展示一个通过多轮对话来补全用户问题的案例：

![](./rag-chat-example.jpg)

但是在实际的生产中，一方面用户可能没有那么大的耐性反复提供程序需要的信息，另一方面开发者也需要考虑如何终止信息采集对话，比如让大模型输出停止语“<EOS>（End Of Sentence）”。所以在实践中，我们需要采用一些更容易实现的方案。

    - 让大模型转述用户问题，再进行RAG问答。参考“指令提示词”的思路，我们可以让大模型来转述用户的问题，将用户的问题标准化，规范化。这里我们可以提供一套标准提示词模板，提供一些标准化的示例，也可以用知识库来增强。我们的主要目的是规范用户的输入请求，再生成RAG查询指令，从而提升RAG查询质量。

![](./rag-qfill.jpg)
图10：让大模型根据知识库来补全用户请求

    - 让用户补全信息辅助业务调用。有一些应用场景需要大量的参数支撑，（比如订火车票需要起点、终点、时间、座位等级、座位偏好等等），我们还可以进一步完善上面的思路，一次性告诉用户系统需要什么信息，让用户来补全。首先，需要准确理解用户的意图，实现意图识别的手段很多，如使用向量相似度匹配、使用搜索引擎、或者直接大模型来支持。其次，根据用户意图选择合适的业务需求模板。接着，让大模型参考业务需求模版来生成一段对话发给用户，请求用户补充信息。这时，如果用户进行了信息完善，那么大模型就可以基于用户的回复信息结合用户的请求来生成下一步的行动指令，整个系统就可以实现应用系统自动帮助用户订机票、订酒店，完成知识库问答等应用形式。

![](./rag-info-fill.jpg)
图11：一个完整的用户信息补全流程示意图

Enrich的方法介绍了一种大模型向用户多次确认需求来补全用户想法的做法。自此，我们假设已经获得了补全过的用户需求，但是由于用户面对的现实问题千变万化，而系统或RAG的知识可能会滞后，对用户问题的理解多少存在一些偏差，我们还可以继续对整个系统进行强化，接下来我们继续介绍“如何让系统更好地理解用户的问题”。

- **Multi-Query 多路召回**

多路召回的方法不是让大模型进行一次改写然后反复向用户确认，而是让大模型自己解决如何理解用户的问题。所以我们首先一次性改写出多种用户问题，让大模型根据用户提出的问题，从多种不同角度去生成有一定提问角度或提问内容上存在差异的问题。让这些存在差异的问题作为大模型对用户真实需求的猜测，然后再把每个问题分别生成答案，并总结出最终答案。

例如：用户问“烤鸭店在哪里？”，大模型会生成：

![](./rag-answer.jpg)

以下就是能生成多路召回策略的提示词模板，你可以在你的项目里直接使用这段提示词模板，其中{question}就是用户输入的问题，你也可以尝试先翻译成中文再使用：
>1.You are an AI language model assistant. Your task is to generate five
>
>2.different versions of the given user question to retrieve relevant documents from a vector database. By > generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
>
>3.Provide these alternative questions separated by newlines. Original question: {question}

- **RAG-Fusion 过滤融合**
  
在经过多路召回获取了各种语料块之后，并不是将所有检索到的语料块都交给大模型，而是先进行一轮筛选，给检索到的语料块进行去重操作，然后按照与原问题的相关性进行排序，再将语料块打包喂给大模型来生成答案。

![](./rag-fusion.jpg)


经过去重复语料筛选，节省了传递给大模型的tokens数量，再经过排序，将更有价值的语料块传递给大模型，从而提升答案的生成质量。
用我们的例子讲就是，志愿者先从多种角度来理解用户的问题，然后对每个问题都去检索资料，查找有用信息，最后把重复信息去掉，再将获取的资料排序。这就能锁定比较接近用户问题的几段语料了，比如“全聚德烤鸭店的地址”，“天外天烤鸭店的地址”，“郭林家常菜店铺的地址”等等，以及这些烤鸭店分布在后海的那些区域，如何步行走过去等等。

- **Step Back 问题摘要**

让大模型先对问题进行一轮抽象，从大体上去把握用户的问题，获得一层高级思考下的语料块。
这个策略的提示词写作

>You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:
>
假如是医疗咨询的场景，用户描述了一大段病情、现象、感受、担忧；或者在法律服务的场景，用户描述了现场情况、事发双方的背景、纠纷的由来等一大段话的时候，我们就可以用这个策略，让大模型先理解一下用户的意图是什么，这个事情大体上看是什么问题。

- **Decomposition 问题分解**
  
这个策略讲究细节，有点像提示词工程中的COT的概念，是把用户的问题拆成一个一个小问题来理解，或者可以说是RAG中的COT。这个策略的提示词如
>1.You are a helpful assistant that generates multiple sub-questions related to an input question. \n
>
>2.The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
>
>3.Generate multiple search queries related to: {question} \n
>
>4.Output (3 queries):
>
使用了这段提示词，大模型生成的子问题如下

![](./decomposition.jpg)

接下来可以使用并行与串行两个策略来执行子任务。并行执行是将每个子任务抛出去获得一个答案，然后再让大模型把所有子任务的答案汇总起来。串行是依次执行子任务，然后将前一个任务生成的答案作为后一个任务的提示词的一部分。这两种执行策略如下图所示

![](./rag-decomposition.jpg)


- **HyDE 假设答案**
  
这个策略让大模型先来根据用户的问题生成一段假设答案，然后用这段假设的答案作为新的问题去文档库里匹配新的文档块，再进行总结，生成最终答案。
好比志愿者听到用户的问题“推荐一家烤鸭店”，第一时间想到了“全聚德烤鸭店不错，我前两天刚吃过！”，接下来，志愿者按照自己的思路找到了全聚德烤鸭店的地址，并给用户讲解如何走过去。

![](./rag-HyDE.jpg)


### 3.4 改造三：改造检索渠道
Corrective Retrieval Augmented Generation (CRAG)是一种改善提取信息质量的策略：如果通过知识库检索得到的信息与用户问题相关性太低，我们就主动搜索互联网，将网络搜索到的信息与知识库搜索到的信息合并，再让大模型进行整理给出最终答案。在工程上我们可以有两种实现方式：
1. 向量相似度，我们用检索信息得到的向量相似度分来判断。判断每个语料块与用户问题的相似度评分，是否高过某个阈值，如果搜索到的语料块与用户问题的相似度都比较低，就代表知识库中的信息与用户问题不太相关；
2. 直接问大模型，我们可以先将知识库检索到的信息交给大模型，让大模型自主判断，这些资料是否能回答用户的问题。

![](./crag.jpg)
图12：CRAG原理图

采用这个搜索策略，当志愿者遇到一个问题，而手边的资料都不能解答这个问题时，志愿者可以上网搜索答案。比如游客问：“天安门升旗仪式是几点钟？”志愿者可能会打开电脑，搜索一下明天天安门升旗仪式的具体时间，然后再回答给游客。这样，至少能让RAG的回答信息的范围有所扩大，回答质量有了提升。
在CRAG的论文中，当面临知识库不完备的情况时，先从互联网下载相关资料再回答的准确率比直接回答的准确率有了较大提升。


### 3.5 改造四：回答前反复思考
Self-RAG，也称为self-reflection，是一种通过在应用中设计反馈路径实现自我反思的策略。基于这个思想，我们可以让应用问自己三个问题：

- 相关性：我获取的这些材料和问题相关吗？
- 无幻觉：我的答案是不是按照材料写的来讲，还是我自己编造的？
- 已解答：我的答案是不是解答了问题？

![](./self-rag.jpg)
图13：Self-RAG原理图

这些判断本身可以通过另一段提示词工程让大模型给出判断，整个项目复杂度有了提升，但回答质量有了保障。
志愿者至少会通过反思这三个问题，在回答游客之前，让答案的质量有所提升。


### 3.6 改造五：从多种数据源中获取资料
这个策略涉及系统性的改造数据的存储和获取环节。传统RAG我们只分析文本文档，我们把文档作为字符串存在向量数据库和文档数据库中。但是现实中的知识还有结构化数据以及图知识库。因此，有很多工作者在研究从数据库中通过NL2SQL的方式直接获取与用户问题相关的数据或统计信息；从GraphDB中用NL2Cypher（显然这是在用Neo4J）获取关联知识。这些方法显然将给RAG带来更多新奇的体验。

![](./rag-search.jpg)
图14：通过大模型搜索数据库来抽取信息

- **从数据库中获取统计指标**
  
大模型可以将用户问题转化为SQL语句去数据库中检索相关信息，这个能力就是NL2SQL（Natural Language to SQL）。如果搜索的问题比较简单，只有单表查询，并获取简单的统计数据如求和、求平均等等，大模型还能稳定地生成正确SQL。如果问题比较复杂涉及多表联合查询，或者涉及复杂的过滤条件，或复杂的排序计算公式，大模型生成SQL的正确性就会下降。我们一般用Spider榜单来评测大模型生成SQL的性能。合理构造提示词调用大模型生成SQL，都可以获得可满足应用的NL2SQL能力。

![](./rag-spider.jpg)
图15：Spider数据集和“执行正确率”评测榜单，榜单上排名靠前的DAIL-SQL+GPT4+Self-Consistency技术，就是使用先检索相似问题构造Few-Shot提示词，再用GPT4来生成SQL，并添加了多路召回策略的方法

- **从知识图谱中获取数据**
  
Neo4j是一款图数据库引擎，可以为我们提供知识图谱构建和计算服务。在知识图谱上，我们调用各种图分析算法，如标签传播、关键节点发现等等，可以快速检索多度关联关系，挖掘隐藏关系。

![](./rag-neo4j.jpg)
图16：将用户的问题转化为Neo4j的Cypher查询语句，从知识图谱中获取关键知识

 

## 总结

![](./rag-conclusion.jpg)
图17：增强RAG能力的多种方法汇总

我们可以通过多种办法来提升RAG的性能，在经典RAG框架之上，可以进行技术改造的方法层出不穷，RAG也是当前最活跃的技术话题之一，我们期待着这个 AI 领域未来会有更大的发展。

### RAG 的流程

- 离线步骤：
  1. 文档加载
  2. 文档切分
  3. 向量化
  4. 灌入向量数据库
     
- 在线步骤：
  1. 获得用户问题
  2. 用户问题向量化
  3. 检索向量数据库
  4. 将检索结果和用户问题填入 Prompt 模版
  5. 用最终获得的 Prompt 调用 LLM
  6. 由 LLM 生成回复

In [29]:
!pip uninstall gradio -y
!pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached gradio-4.36.1-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.7.4-py3-none-any.whl.metadata (109 kB)
INFO: pip is looking at multiple versions of fastapi to determine which version is compatible with other requirements. This could take a while.
  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Using cached starlette-0.37.2-py3-none-any.whl.metadata (5.9 kB)
Using cached gradio-4.36.1-py3-none-any.whl (12.3 MB)
Using cached pydantic-2.7.4-py3-none-any.whl (409 kB)
Using cached fastapi-0.111.0-py3-none-any.whl (91 kB)
Using cached starlette-0.37.2-py3-none-any.whl (71 kB)
  Attempting uninstall: starlette
    Found existing installation: starlette 0.20.4
    Uninstalling starlette-0.20.4:
      Successfully uninstalled starlette-0.20.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.0
    Uninstalling pydantic-1.9.0:
      Successfully uninstalled pydantic-1.9.0
  Attempting uninstall: fastapi
    Found existing i

In [1]:
import gradio as gr

def process_message(message, history, file):
    print(message, history, file)
    # 加载pdf
    # 切 chuck
    # 向量数据库
    # query 转向量
    # 做搜索召回
    # 大模型返回

with gr.Blocks() as demo:
    gr.ChatInterface(process_message, multimodal=True)
    
demo.launch(share=True)

/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/gradio/utils.py:1000: UserWarning: Expected 3 arguments for function <function process_message at 0x7fa57c67f310>, received 2.
  warnings.warn(
/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/gradio/utils.py:1004: UserWarning: Expected at least 3 arguments for function <function process_message at 0x7fa57c67f310>, received 2.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://5c482192ee1fe26db7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/uvicorn/protocols/http/httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/root/miniconda3/envs/aigclass3.8/lib/python3.8/site-packages/st

## 作业

做个自己的 ChatPDF。需求：

1. 从本地加载 PDF 文件，基于 PDF 的内容对话
2. 其它随意发挥
